In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [ ]:
#1. Load Resnet-18
resnet18 = torchvision.models.resnet18(pretrained=True)

In [ ]:
#2 Change the final layer
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 10)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")
resnet18 = resnet18.to(device)

# Thêm vào vòng lặp huấn luyện:
inputs, labels = inputs.to(device), labels.to(device)

Sử dụng thiết bị: cpu


In [ ]:
#3. Prepare the CIFAR-10 data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

In [ ]:
# 4. Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#5. Train the model
for epoch in range(2):
  running_loss= 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    optimizer.zero_grad()
    outputs = resnet18(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i %2000 == 1999:
      print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000)) # Corrected line: Added parentheses to enclose all arguments
      running_loss = 0.0
print('Finished Training')

[1,  2000] loss: 2.645
[1,  4000] loss: 2.460
[1,  6000] loss: 2.368
[1,  8000] loss: 2.224
[1, 10000] loss: 2.157
[1, 12000] loss: 2.171
[2,  2000] loss: 2.033
[2,  4000] loss: 1.979
[2,  6000] loss: 2.094
[2,  8000] loss: 2.072
[2, 10000] loss: 1.984
[2, 12000] loss: 1.861
Finished Training


In [ ]:
#5. Train the model
for epoch in range(5):
  running_loss= 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    optimizer.zero_grad()
    outputs = resnet18(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i %2000 == 1999:
      print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000)) # Corrected line: Added parentheses to enclose all arguments
      running_loss = 0.0
print('Finished Training')

[1,  2000] loss: 0.919
[1,  4000] loss: 0.888
[1,  6000] loss: 0.900
[1,  8000] loss: 0.904
[1, 10000] loss: 0.895
[1, 12000] loss: 0.931
[2,  2000] loss: 0.885
[2,  4000] loss: 0.886
[2,  6000] loss: 0.843
[2,  8000] loss: 0.797
[2, 10000] loss: 0.847
[2, 12000] loss: 0.823
[3,  2000] loss: 0.795
[3,  4000] loss: 0.759
[3,  6000] loss: 0.769
[3,  8000] loss: 0.718
[3, 10000] loss: 0.740
[3, 12000] loss: 0.730
[4,  2000] loss: 0.725
[4,  4000] loss: 0.691
[4,  6000] loss: 0.706
[4,  8000] loss: 0.704
[4, 10000] loss: 0.692
[4, 12000] loss: 0.700
[5,  2000] loss: 0.701
[5,  4000] loss: 0.739
[5,  6000] loss: 0.726
[5,  8000] loss: 0.683
[5, 10000] loss: 0.685
[5, 12000] loss: 0.815
Finished Training


In [ ]:
# Đánh giá mô hình trên tập test
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Độ chính xác trên tập test: {100 * correct / total}%')

Độ chính xác trên tập test: 46.3%


In [ ]:
# 5. Huấn luyện mô hình
for epoch in range(2):
    running_loss = 0.0  # Khởi tạo biến lưu tổng loss cho từng epoch
    # Lặp qua từng batch của dữ liệu huấn luyện
    for i, data in enumerate(trainloader, 0):
        # Lấy dữ liệu đầu vào (inputs) và nhãn (labels) từ batch hiện tại
        inputs, labels = data

        # Đặt gradient về 0 để không cộng dồn qua các batch
        optimizer.zero_grad()

        # Dự đoán đầu ra bằng mô hình ResNet-18
        outputs = resnet18(inputs)

        # Tính toán hàm mất mát giữa đầu ra dự đoán và nhãn thật
        loss = criterion(outputs, labels)

        # Lan truyền ngược để tính gradient
        loss.backward()

        # Cập nhật các tham số của mô hình dựa trên gradient vừa tính
        optimizer.step()

        # Cộng dồn loss của batch hiện tại vào biến running_loss
        running_loss += loss.item()

        # Mỗi 2000 batch, in ra trung bình loss của 2000 batch vừa qua
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')
